In [3]:
import os
import numpy as np
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
# Assuming prepare_chunks.py is a separate file that you control
from prepare_chunks import read_and_chunk_transcripts
import logging

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize OpenAI client. It's good practice to get API key from environment.
# Ensure OPENAI_API_KEY is set in your environment variables.
try:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    if not client.api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set.")
except ValueError as e:
    logging.error(f"Configuration Error: {e}")
    # Exit or handle gracefully if API key is missing
    exit("Exiting: OpenAI API key is missing. Please set OPENAI_API_KEY environment variable.")


# --- 1. Improve Chunking Strategy (Conceptual Change, implement in prepare_chunks.py) ---
# The way you chunk transcripts is critical. If chunks are too small, they lack context.
# If too large, they might contain irrelevant info diluting the similarity.
# Suggestions for `prepare_chunks.py`:
# A. Fixed size with overlap: e.g., 500 characters per chunk with 100 characters overlap.
#    This helps maintain context across chunk boundaries.
# B. Sentence-based chunking: Ensure chunks don't cut sentences in half.
# C. Paragraph-based chunking: If transcripts are paragraphed, use paragraphs as chunks.
# D. Recursive character text splitter (LangChain concept): Splits by paragraphs, then sentences, then words, etc.
#    This is more advanced but often yields better results.
# Make sure your `read_and_chunk_transcripts` function handles this effectively.
try:
    chunks = read_and_chunk_transcripts('transcripts/')
    if not chunks:
        logging.warning("No chunks read from 'transcripts/'. Ensure files exist and content is processed.")
        # Handle case where no chunks are loaded, e.g., exit or use dummy data
        exit("Exiting: No transcript chunks found. Please check 'transcripts/' directory and prepare_chunks.py.")
    logging.info(f"Successfully loaded {len(chunks)} chunks.")
except Exception as e:
    logging.error(f"Error reading and chunking transcripts: {e}")
    exit("Exiting: Failed to process transcripts. Check 'prepare_chunks.py' and 'transcripts/' directory.")


def embed_texts(texts, model="text-embedding-ada-002", batch_size=50):
    """
    Generates embeddings for a list of texts using OpenAI's embedding API.
    Handles batching and includes basic error handling.
    """
    if not texts:
        return []

    embeddings = []
    # Add a retry mechanism for robustness
    max_retries = 3
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        for attempt in range(max_retries):
            try:
                response = client.embeddings.create(
                    model=model,
                    input=batch
                )
                embeddings.extend([item.embedding for item in response.data])
                break # Break out of retry loop on success
            except Exception as e:
                logging.warning(f"Embedding API error on batch {i//batch_size + 1}, attempt {attempt + 1}: {e}")
                if attempt == max_retries - 1:
                    logging.error(f"Max retries reached for embedding batch {i//batch_size + 1}. Skipping batch.")
                    # Optionally, append dummy embeddings or raise the error
                # Add a small delay before retrying
                import time
                time.sleep(1 + attempt * 2) # Exponential backoff
    return embeddings

logging.info("Generating embeddings for chunks...")
chunk_embeddings = embed_texts(chunks)
if not chunk_embeddings:
    logging.error("Failed to generate embeddings for chunks. Check API key and network.")
    exit("Exiting: No chunk embeddings generated.")
logging.info(f"Generated {len(chunk_embeddings)} chunk embeddings.")


def answer_query(query, top_k=5, llm_model="gpt-4o", temperature=0.5): # Increased temperature slightly
    """
    Retrieves relevant chunks and uses an LLM to answer the query.
    """
    logging.info(f"Processing query: '{query}'")

    if not query.strip():
        return "Please enter a valid question."

    # Embed the query
    query_embedding = embed_texts([query])
    if not query_embedding:
        logging.error("Failed to generate embedding for the query.")
        return "Sorry, I couldn't process your question at the moment."
    query_embedding = query_embedding[0]

    # Calculate similarities and get top_k indices
    similarities = cosine_similarity([query_embedding], chunk_embeddings)[0]
    # Ensure top_k doesn't exceed available chunks
    effective_top_k = min(top_k, len(chunks))
    top_indices = np.argsort(similarities)[-effective_top_k:][::-1]

    # Combine top chunks into a single context with clear formatting
    # --- 2. Improve Context Formatting ---
    # Give the LLM clear markers for context sections.
    # Emphasize that these are retrieved documents.
    context_parts = []
    for i in top_indices:
        # Include original index for debugging, and a clear separator
        context_parts.append(f"<DOCUMENT_START id={i}>\n{chunks[i]}\n<DOCUMENT_END>")
    context = "\n\n".join(context_parts)

    logging.info(f"\n🔎 Retrieved top {effective_top_k} chunks (indices: {[int(i) for i in top_indices]}):\n{context}\n")


    # --- 3. Refine Prompt Engineering ---
    # Make the prompt more directive, clearly define the AI's role and instructions.
    # Guide it to synthesize information, and be more flexible about "I don't know".
    prompt = f"""You are a helpful and knowledgeable financial literacy assistant. Your task is to answer the user's question ONLY based on the information provided in the following retrieved documents.

**Instructions:**
- Read the provided documents carefully.
- Synthesize the information to answer the question concisely and accurately.
- If the question cannot be answered using *only* the provided documents, or if the documents do not contain enough relevant information, state clearly: "I apologize, but I don't have enough information in my knowledge base to answer that question based on the provided context." Do NOT make up information.
- Maintain a helpful and informative tone.

<RETRIEVED_DOCUMENTS>
{context}
</RETRIEVED_DOCUMENTS>

User Question: {query}

Financial Literacy Assistant's Answer:
"""
    logging.info("Sending query to LLM...")
    try:
        response = client.chat.completions.create(
            model=llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature, # Adjusted from 0.2 to 0.5 for a bit more flexibility
            max_tokens=700, # Increased max_tokens to allow for more comprehensive answers
        )
        answer = response.choices[0].message.content.strip()
        logging.info("LLM response received.")
        return answer
    except Exception as e:
        logging.error(f"Error calling LLM API: {e}")
        return "I'm sorry, I'm having trouble processing your request right now. Please try again later."


if __name__ == "__main__":
    print("Welcome to Financial Literacy Chatbot (short pipeline demo). Type 'exit' to quit.")
    # Add a check to ensure chunks and embeddings are loaded before starting interaction
    if not chunks or not chunk_embeddings:
        print("Chatbot cannot start due to missing data. Please check logs for errors.")
    else:
        while True:
            query = input("\nAsk your question: ")
            if query.lower() == "exit":
                break
            answer = answer_query(query)
            print(f"\n💬 Answer:\n{answer}")



2025-06-26 22:00:02,840 - INFO - Successfully loaded 1827 chunks.
2025-06-26 22:00:02,841 - INFO - Generating embeddings for chunks...


✅ Loaded 1827 chunks from transcripts.


2025-06-26 22:00:03,321 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:03,996 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:04,539 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:04,969 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:05,660 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:06,395 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:07,089 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:07,809 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:08,470 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:09,171 - INFO - HTTP

Welcome to Financial Literacy Chatbot (short pipeline demo). Type 'exit' to quit.


2025-06-26 22:00:41,047 - INFO - Processing query: 'can you summerize khan academy videos?'
2025-06-26 22:00:41,498 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:00:41,694 - INFO - 
🔎 Retrieved top 5 chunks (indices: [986, 836, 1245, 1739, 185]):
<DOCUMENT_START id=986>
 like you said obviously YouTube jumps to mind I suspect that's on this list somewhere but you can learn essentially anything you need in the entire world on YouTube we talked about the library there are free courses the the moocs the what are they the massively open online courses whatever they stand for there's Khan Academy there's you know like you said sure personal finance books are great but you know you kind of run to the end of that pretty quickly but you can then get into psychology lik
<DOCUMENT_END>

<DOCUMENT_START id=836>
are likely to be which is why you should now watch this video here where i explain how you can start to predict them much more reliably i


💬 Answer:
I apologize, but I don't have enough information in my knowledge base to answer that question based on the provided context.


2025-06-26 22:01:30,547 - INFO - Processing query: 'What are the tax implications of withdrawing from a 401(k) before age 59½?'
2025-06-26 22:01:30,895 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:01:31,069 - INFO - 
🔎 Retrieved top 5 chunks (indices: [304, 181, 1708, 1341, 1700]):
<DOCUMENT_START id=304>
can be invested and grows free of income and capital gains tax the only downside to pensions is that you cannot access them until your 55th birthday but when you reach 55 you get access to the funds inside your pension and you can take 25 percent of the value as a tax free lump sum and the other 75 can be drawn down and it's taxed as if it was your own income alternatively if you don't take the 25 tax-free lump sum straight away as soon as you hit 55 you can choose to do it gradually over time s
<DOCUMENT_END>

<DOCUMENT_START id=181>
ns tax you can then draw 25 of that out tax-free and the rest will be taxed at your marginal income 


💬 Answer:
I apologize, but I don't have enough information in my knowledge base to answer that question based on the provided context.


2025-06-26 22:01:53,681 - INFO - Processing query: 'How do annuities work and when might they be a good option for retirement income?'
2025-06-26 22:01:54,078 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-26 22:01:54,240 - INFO - 
🔎 Retrieved top 5 chunks (indices: [1176, 1170, 1179, 1169, 1600]):
<DOCUMENT_START id=1176>
at I've built for my subscribers to visualize their retirement plans but if you are interested in trying this out for yourself again you can find a link to this down in the description with this strategy we've used an annuity provide an income at the start of retirement but as an alternative Tim could set aside part of their pension say a hundred thousand pounds to be invested with the intention of buying an annuity later on in life at say 75 to provide them with a guaranteed income for the rest
<DOCUMENT_END>

<DOCUMENT_START id=1170>
so in effect they only need to plan for the average life expectancy of the group rather t


💬 Answer:
Annuities work by providing a guaranteed income stream during retirement. When you purchase an annuity, you pay a lump sum or series of payments to an insurance company, which in turn provides you with regular payments for a specified period or for the rest of your life. The insurer is able to provide this guaranteed income by pooling the funds of many annuity holders and investing in bonds. They can pay out higher levels of income because they predict the average life expectancy of their group of annuitants and use funds from those who pass away earlier to pay those who live longer.

Annuities might be a good option for retirement income if you are looking for financial stability and a guaranteed income, especially as interest rates rise, which can lead to higher annuity rates. They are particularly beneficial if you are concerned about outliving your savings, as they can provide income for life. However, it's important to note that once an annuity is set up, it lacks flexi